In [8]:
from random import randint, sample, shuffle, random
from math import sqrt
import time
from __future__ import print_function
import copy
from numpy.random import choice
import operator

class SudokuSolver(object):
    def __init__(self, sudoku_size, init_values):
        self.sudoku_size = sudoku_size
        self.init_values = list(init_values)[:]
        self.fixed_positions = []
        self.free_positions = []
        self.population = []
        self.fitness_hash = {}
        self.available_numbers = {}

        for i in range(0, len(self.init_values)):
            if not(self.init_values[i] == str(0) or self.init_values[i] == '.'):
                self.init_values[i] = int(self.init_values[i])
                self.fixed_positions.append(i)
            else:
                self.free_positions.append(i)
                self.init_values[i] = 0
                
        for pos in self.free_positions:
            available_colors = set(range(1,10))
            for i in self.adyacent_indexes(pos):
                if self.init_values[i] in available_colors:
                    available_colors.remove(self.init_values[i])
            self.available_numbers[pos] = available_colors

        #self.printGrid(self.init_values)

    def nodes(self):
        return self.sudoku_size * self.sudoku_size

    def kcolors(self):
        return self.sudoku_size

    def edges(self):
        return (self.nodes() * (self.kcolors()-1 + (self.kcolors()-(sqrt(self.kcolors()))) * 2)) / 2

    def population(self):
        return self.population

    def printGrid(self, solution, printfile=None):
        for i in range(0, 9):
            if printfile is not None:
                print(solution[i*9:i*9+9], file=printfile)
            else:
                print(solution[i*9:i*9+9])

    def row_indexes(self, position):
        return range(int(position/self.sudoku_size)*9,int(position/self.sudoku_size)*9+9)

    def row_index(self, position):
        return int(position/self.sudoku_size)

    def col_index(self, position):
        return position%self.sudoku_size

    def col_indexes(self, position):
        return range(position%self.sudoku_size,73+(position%self.sudoku_size),9)

    def zone_number(self, position):
        row_index = self.row_index(position)
        col_index = self.col_index(position)
        if (row_index % 9 < 3):
            if (col_index % 9 < 3):
                return 0
            elif (col_index % 9 < 6):
                return 1
            else:
                return 2
        elif (row_index % 9 < 6):
            if (col_index % 9 < 3):
                return 3
            elif (col_index % 9 < 6):
                return 4
            else:
                return 5
        else:
            if (col_index % 9 < 3):
                return 6
            elif (col_index % 9 < 6):
                return 7
            else:
                return 8

    def zone_indexes(self, zone_number):
        first = self.sudoku_size * int(zone_number/3)*3 + zone_number % 3 * 3
        indexes = []
        for i in range(0, 3):
            indexes.append(first) 
            indexes.append(1 + first)
            indexes.append(2 + first)
            first = first + self.sudoku_size
        return indexes

    def zone_indexes_by_pos(self, position):
        zone_number = self.zone_number(position)
        return self.zone_indexes(zone_number)

    def adyacent_indexes(self, position):
        indexes = set()
        [indexes.add(x) for x in self.col_indexes(position)]
        [indexes.add(x) for x in self.row_indexes(position)]
        [indexes.add(x) for x in self.zone_indexes_by_pos(position)]
        indexes.remove(position)
        return indexes

    def reorderPopulation(self):
        new_fitness_hash = {}
        for solution in self.population:
            if self.sol2string(solution) in self.fitness_hash:
                new_fitness_hash[self.sol2string(solution)] = self.fitness_hash[self.sol2string(solution)]
            else:
                bad_edges = 0
                for pos in self.free_positions:
                    for ady in self.adyacent_indexes(pos):
                        if (solution[ady] == solution[pos]):
                            bad_edges += 1
                new_fitness_hash[self.sol2string(solution)] = bad_edges / 2

        # Envejezco al mejor
        if len(self.fitness_hash) > 0:
            best_old = max(self.fitness_hash.items(), key=operator.itemgetter(1))[0]
            best_new = max(new_fitness_hash.items(), key=operator.itemgetter(1))[0]
            if best_old == best_new:
                new_fitness_hash[self.sol2string(best_old)] += 1

        self.fitness_hash = new_fitness_hash
        self.population = sorted(self.population, key=self.fitness)

    def fitness(self, solution):
        return int(self.fitness_hash[self.sol2string(solution)])

    def sol2string(self, sol):
        s = ''
        for i in sol:
            s += str(i)
        return s

    def initPopulation(self, n_population):
        self.population = []
        for i in range(0, n_population):
            solution = self.createRandomSol()
            self.population.append(copy.copy(solution))

    def createRandomSol(self):
        solution = self.init_values
        indexes = copy.copy(self.free_positions)
        shuffle(indexes)
        while (len(indexes) > 0):
            i = indexes.pop()
            solution[i] = self.getAvailableNumber(i)
        return solution
    
    def getAvailableNumber(self, pos):
        return sample(self.available_numbers[pos], 1)[0]

    def select_parents(self, tournament_size = 5):
        n_population = len(self.population)

        participants = []
        for i in range(0, tournament_size):
            new = randint(0, int(n_population/2)-1)
            while new in participants:
                new = randint(0, int(n_population/2)-1)
            participants.append(new)

        participants = sorted(participants)
        parent1_index = participants[0]
        parent2_index = participants[1]
        
        return self.population[parent1_index], self.population[parent2_index]

    def crossover(self, parent1, parent2):
        # Por zonas
        solution = [0] * 81
        for zone_number in range(0,9):
            r = randint(0,1)
            if (r):
                parent = parent1
            else:
                parent = parent2
            indexes = self.zone_indexes(zone_number)
            for i in indexes:
                solution[i] = parent[i]
        return solution

    def mutation(self, solution, normal_proba=0.1, bad_colored_proba=0.9):
        indexes = copy.copy(self.free_positions)
        shuffle(indexes)
        while (len(indexes) > 0):
            i = indexes.pop()
            r = random()

            bad_colored = False

            for ady in self.adyacent_indexes(i):
                if solution[i] == solution[ady]:
                    bad_colored = True
                    break

            if bad_colored and r <= bad_colored_proba:
                solution[i] = self.getAvailableNumber(i)

            elif r <= normal_proba:
                solution[i] = self.getAvailableNumber(i)

        return solution

    def solve(self,
              n_population = 200,
              reset_point = 300,
              max_iters = 50000,
              crossover_proba = 0.9,
              selection_tournament_size = 5,
              normal_mutation_proba = 0.1,
              bad_colored_mutation_proba = 0.9,
              elitism_ratio = 0.5
             ):
        
        # Inicializacion de variables
        start_time = time.time()
        self.initPopulation(n_population)
        best_fitness = self.edges()
        iter_count = 0
        iter_without_improvement = 0
        elite_size = int(n_population * elitism_ratio)

        while (best_fitness > 0 and iter_count < max_iters):

            # Actualizacion del best fitness
            self.reorderPopulation()
            new_best = self.fitness(self.population[0])

            if (new_best < best_fitness):
                best_fitness = new_best
                iter_without_improvement = 0
                #print("iteration %d, new best_fitness %d" %(iter_count, best_fitness))
                #self.printGrid(self.population[0])
                if best_fitness == 0: 
                    break
            else:
                iter_without_improvement += 1

            # Restarts, para destrabar y resetear el algoritmo
            if iter_without_improvement > reset_point:
                print("Restarteando")
                self.initPopulation(n_population)
                best_fitness = self.edges()
                iter_without_improvement = 0

            else:
                # Actualizacion de la poblacion
                new_population = []
                for i in range(0, elite_size):
                    new_population.append(self.population[i])
                
                start_new_popu = time.time()

                for i in range(elite_size, n_population):
                    # Generacion del hijo, evito que se generen repetidos
                    hijo_repetido = True
                    intentos = 0
                    while (hijo_repetido):

                        # Seleccion de padres (tournament selection)
                        parent1, parent2 = self.select_parents(tournament_size=selection_tournament_size)

                        # Crossover

                        if random() <= crossover_proba:
                            child = self.crossover(parent1, parent2)
                        else:
                            child = copy.copy(parent1)

                        # Mutaciones
                        child = self.mutation(child,
                                              normal_proba = normal_mutation_proba,
                                              bad_colored_proba = bad_colored_mutation_proba)

                        # Chequeo si es repetido
                        hijo_repetido = False
                        for p in self.population :  
                            if p == child :
                                hijo_repetido = True
                                intentos += 1
                                break

                        # Si pasa el nro max de intentos repetidos genero un hijo random        
                        if intentos > 3 :
                            print("Creando Random")
                            child = self.createRandomSol()
                            hijo_repetido = False

                        if not(hijo_repetido):
                            new_population.append(copy.copy(child))

                #print("Tiempo new popu %s segundos" % (time.time() - start_new_popu))
                self.population = new_population

            iter_count = iter_count + 1

        # Resultado
        if (best_fitness == 0):
            print("--- Se resolvió el SUDOKU en %s segundos y %d iteraciones !!!! ---" % (time.time() - start_time, iter_count))
            #self.printGrid(self.population[0])
            return True, iter_count, time.time() - start_time
        else:
            print("NO ANDUVO, se obtuvo un fitness de %d luego de %s segundos y %d iteraciones :(" % (best_fitness, (time.time() - start_time), iter_count))
            return False, 0, time.time() - start_time

In [2]:
def test_sudoku_results(grid):
    s = SudokuSolver(9, grid)
    
    solved_count = 0
    solved_iters = []
    times = []

    for i in range(0, 100):
        solved, iters, time_elapsed = s.solve()
        if solved:
            solved_count += 1
            solved_iters.append(iters)
            times.append(time_elapsed)
    
    return solved_count, solved_iters, times

In [3]:
# Set de testeo para comparar con los papers
stars1 = '040000179002008054006005008080070910050090030019060040300400700570100200928000060'
stars2 = '206000049037009000100700006000580900705000804009062000900004001000300490410000208'
stars3 = '050200000300005080960078200000030020708000103040080000001640032070500001000009050'
stars4 = '050090000004800009000107280560000137000000000173000042021508000600003800000010060'
stars5 = '105000370000000200097300010000053102300801004201470000070008640008000000012000807'
easy = '008060900000203678706051004973048100620039050001700000580900306000000000040005721'
challenging = '002004000008600000090000030800062000140030509070000000035008607000000042000901003'
difficult = '670008010020060000000030000201000006480001700000000009004500000000000300003400802'
sdifficult = '090384000002070000000000071500003240030000000001005090000800000706520000000006400'

In [4]:
import numpy as np

def mostrarMedidas(count, iters, times):
    print("Count \t %d" % count)
    print("Min \t %d" % min(iters))
    print("Max \t %d" % max(iters))   
    print("Med \t %d" % np.median(iters))            
    print("Avg \t %f" % np.mean(iters))        
    print("Std \t %f" % np.std(iters))
    print("Time \t %f" % np.mean(times))

In [399]:
count1, iters1, times1 = test_sudoku_results(stars1)
mostrarMedidas(count1, iters1, times1)

--- Se resolvió el SUDOKU en 3.90260314941 segundos y 20 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 6.08791780472 segundos y 22 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 4.70534515381 segundos y 16 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 4.2951939106 segundos y 21 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 5.47411799431 segundos y 22 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 4.65733790398 segundos y 21 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 4.36898684502 segundos y 18 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 6.33577179909 segundos y 22 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 4.2915968895 segundos y 16 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 5.44368100166 segundos y 20 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 6.05939888954 segundos y 21 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 3.75098490715 segundos y 18 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 5.58781695366 segundos y 22 iteracion

In [ ]:
count2, iters2, times2 = test_sudoku_results(stars2)
mostrarMedidas(count2, iters2, times2)

In [ ]:
count3, iters3, times3 = test_sudoku_results(stars3)
mostrarMedidas(count3, iters3, times3)

In [ ]:
count4, iters4, times4 = test_sudoku_results(stars4)
mostrarMedidas(count4, iters4, times4)

In [ ]:
count5, iters5, times5 = test_sudoku_results(stars5)
mostrarMedidas(count5, iters5, times5)

In [400]:
counte, iterse, timese = test_sudoku_results(easy)
mostrarMedidas(counte, iterse, timese)

--- Se resolvió el SUDOKU en 2.38646197319 segundos y 10 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 1.76511311531 segundos y 10 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 1.83873486519 segundos y 10 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 2.37618899345 segundos y 12 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 2.08451700211 segundos y 8 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 3.84988117218 segundos y 11 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 2.63499498367 segundos y 9 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 2.827917099 segundos y 14 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 1.98352503777 segundos y 9 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 4.07909297943 segundos y 13 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 2.35358405113 segundos y 11 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 1.79387807846 segundos y 8 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 2.57827711105 segundos y 13 iteraciones !

In [401]:
countc, itersc, timesc = test_sudoku_results(challenging)
mostrarMedidas(countc, itersc, timesc)

--- Se resolvió el SUDOKU en 76.6203410625 segundos y 314 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 52.8347821236 segundos y 220 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 96.0204479694 segundos y 387 iteraciones !!!! ---
Restarteando
--- Se resolvió el SUDOKU en 251.699048042 segundos y 1039 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 52.7551620007 segundos y 213 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 68.4363689423 segundos y 277 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 115.350200891 segundos y 486 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 73.4662292004 segundos y 298 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 81.0048451424 segundos y 325 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 95.4023308754 segundos y 405 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 107.27257514 segundos y 439 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 110.485905886 segundos y 454 iteraciones !!!! ---
--- Se resolvió el SUDOKU en 62.5176310

In [402]:
countd, itersd, timesd = test_sudoku_results(difficult)
mostrarMedidas(countd, itersd, timesd)

Restarteando
Restarteando
Restarteando
Restarteando
--- Se resolvió el SUDOKU en 1514.03490186 segundos y 5610 iteraciones !!!! ---
Restarteando
Restarteando
--- Se resolvió el SUDOKU en 764.016508818 segundos y 3178 iteraciones !!!! ---
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
--- Se resolvió el SUDOKU en 2094.40704203 segundos y 9290 iteraciones !!!! ---
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
--- Se resolvió el SUDOKU en 2179.54015017 segundos y 9920 iteraciones !!!! ---
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
--- Se resolvió el SUDOKU en 978.353760004 segundos y 4498 iteraciones !!!! ---
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Restarteando
Re

KeyboardInterrupt: 

In [ ]:
countsd, iterssd, timessd = test_sudoku_results(sdifficult)
mostrarMedidas(countsd, iterssd, timessd)